In [ ]:
import joblib
import pickle
import pandas as pd
import numpy as np
import math
import tensorflow as tf
import keras
import time
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Conv2D, MaxPool2D
from keras import backend as K
from keras.layers import BatchNormalization
from keras.initializers import glorot_uniform
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
# from scikeras.wrappers import KerasClassifier
from keras import metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
from keras.regularizers import l2
from keras.initializers import HeNormal
from tensorflow.keras.utils import to_categorical

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/FingerClassification/Data/data_final1.pkl')

In [ ]:
df.head()

,Participant,Handedness,Finger,Task,Matrix,Timestamp,BlobCoords,BlobCount,Blobs,Resized_Blobs_OpenCV,BlobImage,BlobImgFlattened,Resized_Blobs,Scaled_Blobs,Major_Axis,Minor_Axis,Area,Angle
24,1,left,thumb,DRAG,"[[0, 1, 1, 1, 1, 1, 0, -1, 2, 157, 3, 3, 2, 1,...",1508231878417,"[(3, 8, 16, 21)]",1,"[[1, 4, 7, 7, 4], [1, 13, 62, 67, 13], [5, 71,...","[[1.0, 1.0, 1.5833333333333335, 2.416666666666...","[[1, 4, 7, 7, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[1, 4, 7, 7, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[1.0, 2.3125, 4.1875, 6.0625, 7.0, 7.0, 5.312...","[[1, 1, 1, 1, 3, 3, 4, 5, 6, 6, 5, 5, 4, 3, 4,...",11.249028,15.156587,133.907929,92.469559
23,1,left,thumb,DRAG,"[[0, 1, 1, 1, 1, 1, 0, -1, 2, 157, 3, 3, 2, 1,...",1508231878417,"[(3, 8, 16, 21)]",1,"[[1, 4, 7, 7, 4], [1, 13, 62, 67, 13], [5, 71,...","[[1.0, 1.0, 1.5833333333333335, 2.416666666666...","[[1, 4, 7, 7, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[1, 4, 7, 7, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[1.0, 2.3125, 4.1875, 6.0625, 7.0, 7.0, 5.312...","[[1, 1, 1, 1, 3, 3, 4, 5, 6, 6, 5, 5, 4, 3, 4,...",10.452435,13.830733,113.540955,111.972496
25,1,left,thumb,DRAG,"[[0, 1, 1, 1, 1, 1, 0, -1, 2, 157, 3, 3, 2, 1,...",1508231878487,"[(3, 8, 16, 21)]",1,"[[0, 4, 5, 5, 0], [1, 16, 67, 80, 15], [2, 75,...","[[-2.0, -2.0, -0.8333333333333331, 0.833333333...","[[0, 4, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[0, 4, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.0, 1.75, 4.0625, 4.6875, 5.0, 5.0, 2.1875,...","[[0, 0, 0, 1, 1, 3, 4, 5, 5, 4, 3, 2, 1, 0, 0,...",9.565728,11.702647,87.920881,45.661560
27,1,left,thumb,DRAG,"[[0, 1, 1, 1, 1, 1, 0, -1, 2, 157, 3, 3, 2, 1,...",1508231878557,"[(3, 8, 16, 21)]",1,"[[0, 4, 9, 7, 2], [2, 16, 71, 83, 15], [1, 80,...","[[0.0, 0.0, 0.7777777777777779, 1.888888888888...","[[0, 4, 9, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[0, 4, 9, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.0, 1.75, 4.3125, 7.4375, 8.375, 7.125, 4.1...","[[0, 0, 0, 0, 1, 3, 4, 5, 5, 6, 6, 5, 4, 3, 2,...",10.345570,12.890965,104.744140,47.160843
26,1,left,thumb,DRAG,"[[0, 1, 1, 1, 1, 1, 0, -1, 2, 157, 3, 3, 2, 1,...",1508231878557,"[(3, 8, 16, 21)]",1,"[[0, 4, 9, 7, 2], [2, 16, 71, 83, 15], [1, 80,...","[[0.0, 0.0, 0.7777777777777779, 1.888888888888...","[[0, 4, 9, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[0, 4, 9, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.0, 1.75, 4.3125, 7.4375, 8.375, 7.125, 4.1...","[[0, 0, 0, 0, 1, 3, 4, 5, 5, 6, 6, 5, 4, 3, 2,...",9.804294,14.024021,107.988808,42.690639


In [ ]:
df = df.sort_values(by='Timestamp', ascending=True)

In [ ]:
le = LabelEncoder()
# Đặt những người tham gia vào tập kiểm tra
split_factor = 0.8
ptcp_train = df.Participant.unique()[:math.floor(len(df.Participant.unique())*split_factor)]
ptcp_test = df.Participant.unique()[math.floor(len(df.Participant.unique())*split_factor):]
df_train = df[df.Participant.isin(ptcp_train)].copy(deep=True)
df_test = df[df.Participant.isin(ptcp_test)].copy(deep=True)

In [ ]:
max_value = np.max([np.max(x) for x in df['BlobImgFlattened'] if isinstance(x, (list, np.ndarray))])
min_value = np.min([np.min(x) for x in df['BlobImgFlattened'] if isinstance(x, (list, np.ndarray))])
print(max_value, min_value)

268 0


In [ ]:
df['BlobImgFlattened'] =df['BlobImgFlattened'] / 268

In [ ]:
max_value = np.max([np.max(x) for x in df['BlobImgFlattened'] if isinstance(x, (list, np.ndarray))])
print(max_value, min_value)

In [ ]:
df['BlobImgFlattened'].iloc[0].shape

(405,)

In [ ]:
df_test.Participant.unique()

array([17, 18, 19, 20])

In [ ]:
len(df)

95571

In [ ]:
# Change the column to take as the feature (i.e., Finger, Handedness)
feature_column = "Handedness"

In [ ]:
# Training set: Convert target names into numbers for classification
X = ""
y = ""
target_names = df[feature_column].unique()
for l in target_names:
    df_temp = df_train[(df_train[feature_column] == l)]
    X_temp = [df_temp.BlobImgFlattened.iloc[i] for i in range(len(df_temp))]
    y_temp = np.ones(len(X_temp)) * np.where(target_names == l)[0][0]

    if (type(X) is str):
        X = X_temp
        y = y_temp
    else:
        X.extend(X_temp)
        y = np.append(y, y_temp)
train_x = np.array(X)
train_y = np.array(y)


# Test set: Convert target names into numbers for classification
X = ""
y = ""
for l in target_names:
    df_temp = df_test[(df_test[feature_column] == l)]
    X_temp = [df_temp.BlobImgFlattened.iloc[i] for i in range(len(df_temp))]
    y_temp = np.ones(len(X_temp)) * np.where(target_names == l)[0][0]

    if (type(X) is str):
        X = X_temp
        y = y_temp
    else:
        X.extend(X_temp)
        y = np.append(y, y_temp)

test_x = np.array(X)
test_y = np.array(y)

In [ ]:
img_rows, img_cols = 27, 15

In [ ]:
len(df_train[feature_column].unique())

2

In [ ]:
(x_train, y_train), (x_test, y_test) = (train_x, train_y), (test_x, test_y)

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:

model = Sequential([
    Conv2D(64, kernel_size=(3, 3), padding='Same', activation='relu', input_shape=input_shape),
    BatchNormalization(axis=-1),
    Conv2D(64, kernel_size=(3, 3), padding='Same', activation='relu'),
    BatchNormalization(axis=-1),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.5),
    Conv2D(128, kernel_size=(3, 3), padding='Same', activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(128, kernel_size=(3, 3), padding='Same', activation='relu'),
    BatchNormalization(axis=-1),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.5),
    Flatten(),
    Dense(256, activation="relu", kernel_initializer=glorot_uniform()),
    BatchNormalization(axis=-1),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/FingerClassification/Model/Thumb/model_cnn1.h5', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', patience = 5, verbose=0)

history = model.fit(x_train, y_train,
                    validation_data=(x_test, y_test),
                    epochs=100, batch_size=50, callbacks=[checkpoint, early_stopping])


In [ ]:
model.evaluate(x_test, y_test, verbose = 1)

620/620 [==============================] - 38s 62ms/step - loss: 0.2188 - accuracy: 0.9095


[0.21881484985351562, 0.9094668626785278]

In [ ]:
import numpy as np
import time

# Giả sử bạn đã tải mô hình và dữ liệu x_test vào

# Đo thời gian suy luận cho 100 mẫu dữ liệu đầu tiên
inference_times = []
start_time1 = time.time()
for i in range(100):
    sample = np.expand_dims(x_test[i], axis=0)  # Đảm bảo mẫu có đúng số chiều
    start_time = time.time()
    _ = model.predict(sample)
    end_time = time.time()
    inference_times.append(end_time - start_time)
end_time1 = time.time()
inference_time1 = end_time1 - start_time1
# Tính toán min, max, và trung bình của thời gian suy luận
min_inference_time = np.min(inference_times)
max_inference_time = np.max(inference_times)
average_inference_time = np.mean(inference_times)

min_inference_time, max_inference_time, average_inference_time, inference_time1

1/1 [==============================] - 0s 24ms/step


(0.0759584903717041,
 0.161393404006958,
 0.10707566261291504,
 10.715490818023682)